In [19]:
import pyvista as pv
import vtk
import numpy as np

In [20]:
points = np.array([
    [0.5, 0.0, 0.0],
    [1.0, 0.0, 0.0],
    [1.0, 1.0, 0.0],
    [0.5, 1.0, 0.0],
])

# 2) faces 배열 정의
#    첫 원소 '4'는 꼭짓점 개수, 그 뒤로 점 인덱스
faces = np.hstack([[4, 0, 1, 2, 3]])
# 3) PolyData 생성
square = pv.PolyData(points, faces)
mesh = square.extrude_rotate(
    angle=270,
    capping=True,
    resolution=30,
    rotation_axis=(1,0,0)
)
mesh.plot()

Widget(value='<iframe src="http://localhost:39811/index.html?ui=P_0x7f087169fee0_4&reconnect=auto" class="pyvi…

In [21]:
resolution = 50

xmin, xmax, ymin, ymax, zmin, zmax = mesh.bounds

margin = 0.1
x_vals = np.linspace(xmin - margin, xmax + margin, resolution)
y_vals = np.linspace(ymin - margin, ymax + margin, resolution)
z_vals = np.linspace(zmin - margin, zmax + margin, resolution)

X, Y, Z = np.meshgrid(x_vals, y_vals, z_vals, indexing='ij')
random_points = np.column_stack([X.ravel(), Y.ravel(), Z.ravel()])
# random_points = np.random.rand(resolution, resolution, resolution).ravel(order='F')
grid = pv.ImageData(dimensions=(resolution, resolution, resolution))
# grid.origin = (0.0, 0.0, 0.0)
# grid.spacing = (1.0, 1.0, 1.0)
grid.origin = (xmin, ymin, zmin)
grid.spacing = (x_vals[1] - x_vals[0], y_vals[1] - y_vals[0], z_vals[1] - z_vals[0])
grid.point_data['points'] = random_points
grid.set_active_scalars('points')
grid.plot()

Widget(value='<iframe src="http://localhost:39811/index.html?ui=P_0x7f07c99e5420_5&reconnect=auto" class="pyvi…

In [22]:
help(grid.compute_implicit_distance)

Help on method compute_implicit_distance in module pyvista.core.filters.data_set:

compute_implicit_distance(surface, inplace=False) method of pyvista.core.grid.ImageData instance
    Compute the implicit distance from the points to a surface.
    
    This filter will compute the implicit distance from all of the
    nodes of this mesh to a given surface. This distance will be
    added as a point array called ``'implicit_distance'``.
    
    Nodes of this mesh which are interior to the input surface
    geometry have a negative distance, and nodes on the exterior
    have a positive distance. Nodes which intersect the input
    surface has a distance of zero.
    
    Parameters
    ----------
    surface : pyvista.DataSet
        The surface used to compute the distance.
    
    inplace : bool, default: False
        If ``True``, a new scalar array will be added to the
        ``point_data`` of this mesh and the modified mesh will
        be returned. Otherwise a copy of this mesh

In [23]:
sdf_grid = grid.compute_implicit_distance(surface=mesh)
sdf_grid.set_active_scalars('implicit_distance')
sdf_grid.plot()

Widget(value='<iframe src="http://localhost:39811/index.html?ui=P_0x7f07c7b26a70_6&reconnect=auto" class="pyvi…

In [24]:
surface = sdf_grid.contour(isosurfaces=1, scalars='implicit_distance', rng=[-0.1, 0.1], method='marching_cubes')
surface.plot()

Widget(value='<iframe src="http://localhost:39811/index.html?ui=P_0x7f07c810e620_7&reconnect=auto" class="pyvi…